## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_1 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o1/o1_bincore6.csv", index_col=0)
print(bin6_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_1.reset_index(inplace=True, drop=True)

print('shape -', bin6_1.shape)
print('reset_index 완료')
print('input data shape')
bin6_1.head()

(27602871, 2)
shape - (27602871, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_1['label'].value_counts())

256
0    27564156
1       38715
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_1[bin6_1['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_1), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_1', len(ls_idx_bin))

# loc 로 수정필요
bin6_1_Ngram = bin6_1.loc[ls_idx_bin,:].copy()

619440
나머지 0
최종 길이 619440
bin6_1 619440


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_1_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_1.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_1[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

38715.0
0 13108475
1000 12523182
2000 7971785
3000 14443018
4000 23748156
5000 8693184
6000 4343379
7000 4012078
8000 27345081
9000 8845372
10000 16617843
11000 21656864
12000 9014492
13000 23163402
14000 6351336
15000 1317757
16000 6535197
17000 13035845
18000 3221986
19000 19245359
20000 18294894
21000 15496517
22000 22558664
23000 14120864
24000 13350541
25000 7718840
26000 14061155
27000 10222188
28000 2515980
29000 26140578
30000 21873504
31000 18956503
32000 13939093
33000 17852161
34000 9957823
35000 17265241
36000 25049659
37000 7762825
38000 11006127
완료
38715


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_1['label'].value_counts()[1])

38715
38715


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_1_Ngram])
final.shape

(1238880, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_1_onehot_Ngram = pd.concat([final['label'], bc6_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_1_onehot_Ngram.shape)

원핫인코딩완료
(1238880, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_1 = bc6_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_1 = bc6_1_onehot_Ngram['label'].to_numpy()
print(x_bc6_1.shape, x_bc6_1.shape)

x_bc6_1 = x_bc6_1.reshape(-1, right_idx, x_bc6_1.shape[1])
y_bc6_1 = y_bc6_1.reshape(-1, right_idx, 1)

print(x_bc6_1.shape, y_bc6_1.shape)

(1238880, 256) (1238880, 256)
(77430, 16, 256) (77430, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_1.shape[0])

x_bc6_1 = x_bc6_1[p]
y_bc6_1 = y_bc6_1[p]

print(x_bc6_1.shape, y_bc6_1.shape)

(77430, 16, 256) (77430, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_1, y_bc6_1):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_1[train],
               y_bc6_1[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_1[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_1[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 27s 386us/sample - loss: 0.0303 - accuracy: 0.9923
Epoch 2/10
69687/69687 [==============================] - 23s 336us/sample - loss: 0.0057 - accuracy: 0.9983
Epoch 3/10
69687/69687 [==============================] - 23s 333us/sample - loss: 0.0041 - accuracy: 0.9988
Epoch 4/10

accuracy_score 0.9993865426837143
recall_score 0.9918951132300358
precision_score 0.9900071377587437
f1_score 0.9909502262443439
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 27s 391us/sample - loss: 0.0014 - accuracy: 0.9996
Epoch 2/10
69687/69687 [==============================] - 23s 327us/sample - loss: 0.0013 - accur

accuracy_score 0.9995721942399587
recall_score 0.9943957115009746
precision_score 0.9927025054731209
f1_score 0.9935483870967742
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 37s 528us/sample - loss: 8.6233e-04 - accuracy: 0.9997
Epoch 2/10
69687/69687 [==============================] - 35s 507us/sample - loss: 7.8610e-04

accuracy_score 0.9996690559214775
recall_score 0.9942390782525204
precision_score 0.9959124789612888
f1_score 0.995075075075075
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 44s 632us/sample - loss: 5.9192e-04 - accuracy: 0.9999
Epoch 2/10
69687/69687 [==============================] - 36s 513us/sample - loss: 5.0883e-04 

accuracy_score 0.9998466356709286
recall_score 0.9972235076353541
precision_score 0.9983785035904563
f1_score 0.9978006713740016
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 43s 621us/sample - loss: 3.9791e-04 - accuracy: 0.9999
Epoch 2/10
69687/69687 [==============================] - 37s 535us/sample - loss: 3.8674e-04

...

accuracy_score 0.9998304920573421
recall_score 0.9966329966329966
precision_score 0.9983136593591906
f1_score 0.9974726200505475
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 41s 595us/sample - loss: 2.0528e-04 - accuracy: 0.9999
Epoch 2/10
69687/69687 [==============================] - 36s 520us/sample - loss: 1.9617e-04

accuracy_score 0.9999273537388609
recall_score 0.9990451181666269
precision_score 0.9988066825775657
f1_score 0.9989258861439313
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 47s 673us/sample - loss: 1.5516e-04 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 41s 594us/sample - loss: 1.2513e-04

accuracy_score 0.9998789228981015
recall_score 0.9967128433904673
precision_score 0.9997644842204427
f1_score 0.9982363315696648
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 47s 670us/sample - loss: 9.7456e-05 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 39s 557us/sample - loss: 9.4637e-05

accuracy_score 0.9999596409660338
recall_score 0.9992841803865425
precision_score 0.9995226730310263
f1_score 0.9994034124806109
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69687 samples
Epoch 1/10
69687/69687 [==============================] - 43s 614us/sample - loss: 8.2228e-05 - accuracy: 1.0000
Epoch 2/10
69687/69687 [==============================] - 38s 539us/sample - loss: 5.4219e-05

accuracy_score 0.9999757845796203
recall_score 0.999526066350711
precision_score 0.9997629770087698
f1_score 0.9996445076430857

K-fold cross validation Accuracy: [0.9993865426837143, 0.9995721942399587, 0.9996690559214775, 0.9998466356709286, 0.9998466356709286, 0.9998304920573421, 0.9999273537388609, 0.9998789228981015, 0.9999596409660338, 0.9999757845796203]

K-fold cross validation Recall: [0.9918951132300358, 0.9943957115009746, 0.9942390782525204, 0.9972235076353541, 0.9978535654662533, 0.9966329966329966, 0.9990451181666269, 0.9967128433904673, 0.9992841803865425, 0.999526066350711]

K-fold cross validation Precision: [0.9900071377587437, 0.9927025054731209, 0.9959124789612888, 0.9983785035904563, 0.9976156413924654, 0.9983136593591906, 0.9988066825775657, 0.9997644842204427, 0.9995226730310263, 0.9997629770087698]

K-fold cross validation F1-Score: [0.9909502262443439, 0.9935483870967742, 0.995075075075075, 0.9978006713740016, 0.9977345892452606, 0.9974726200505475, 0.998925886

## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997893258426966
10-Fold Cross_validation. Recall : 0.9966808181012483
10-Fold Cross_validation. Precision : 0.997078674337307
10-Fold Cross_validation. F1-Score : 0.9968791706923295


In [13]:
model1.save('gcc6_bin_core_s16_h24_o1.h5')
print('save 완료')

save 완료
